# MSM estimation and validation

<a rel="license" href="http://creativecommons.org/licenses/by/4.0/"><img alt="Creative Commons Licence" style="border-width:0" src="https://i.creativecommons.org/l/by/4.0/88x31.png" title='This work is licensed under a Creative Commons Attribution 4.0 International License.' align="right"/></a>

In this notebook, we will cover how to estimate a Markov state model (MSM) and do model validation;
we also show how to save and restore model and estimator objects.
For this notebook, you need to know how to do data loading/visualization as well as dimension reduction.


**Remember**:
- to run the currently highlighted cell, hold <kbd>&#x21E7; Shift</kbd> and press <kbd>&#x23ce; Enter</kbd>;
- to get help for a specific function, place the cursor within the function's brackets, hold <kbd>&#x21E7; Shift</kbd>, and press <kbd>&#x21E5; Tab</kbd>;
- you can find the full documentation at [PyEMMA.org](http://www.pyemma.org).

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import numpy as np
import mdshare
import pyemma
import deeptime as dt

## Loading MD data and repeating the clustering step

Let's load alanine dipeptide backbone torsions and discretise with 200 $k$-means centers...

In [ ]:
pdb = mdshare.fetch('alanine-dipeptide-nowater.pdb', working_directory='data')
files = mdshare.fetch('alanine-dipeptide-*-250ns-nowater.xtc', working_directory='data')

feat = pyemma.coordinates.featurizer(pdb)
feat.add_backbone_torsions(periodic=False)

data = pyemma.coordinates.load(files, features=feat)
data_concatenated = np.concatenate(data)

cluster = dt.clustering.Kmeans(200, max_iter=50).fit(data_concatenated[::10]).fetch_model()

... and plot the free energy along with the cluster centers:

In [ ]:
fig, ax = plt.subplots()
pyemma.plots.plot_free_energy(*data_concatenated.T, ax=ax, legacy=False)
ax.scatter(*cluster.cluster_centers.T, s=15, c='k')
ax.set_xlabel('$\Phi$ / rad') 
ax.set_ylabel('$\Psi$ / rad')
fig.tight_layout()

## Implied time scales and lag time selection

The first step after obtaining the discretized dynamics is finding a suitable lag time.
The systematic approach is to estimate MSMs at various lag times and observe how the implied timescales (ITSs) of these models behave.
In particular, we are looking for lag time ranges in which the implied timescales are constant
To this aim, PyEMMA provides the `its()` function which we use to track the first four (`nits=4`) implied timescales:

In [ ]:
dtrajs = [cluster.transform(traj) for traj in data]

lags = [1, 2, 5, 10, 20, 50]
timescales = []
for lag in tqdm(lags):
    counts_estimator = dt.markov.TransitionCountEstimator(lag, "sliding")
    counts = counts_estimator.fit(dtrajs).fetch_model()
    counts = counts.submodel_largest()
    
    msm_estimator = dt.markov.msm.MaximumLikelihoodMSM()
    msm = msm_estimator.fit(counts).fetch_model()
    timescales.append(msm.timescales(k=4))

$$\begin{eqnarray*}
T(n \tau) & = & (T(\tau))^n\\[0.75em]
\lambda(n \tau) & = & (\lambda(\tau))^n\\[0.75em]
\mathrm{ITS}(n \tau) & = & - \frac{n \tau}{\ln \lambda(n \tau)} = - \frac{n \tau}{\ln (\lambda(\tau))^n} = - \frac{\tau}{\ln \lambda(\tau)} = \mathrm{ITS}(\tau)
\end{eqnarray*}$$

We can pass the returned `its` object to the `pyemma.plots.plot_implied_timescales()` function:

In [ ]:
pyemma.plots.plot_implied_timescales((lags, timescales), units='ps');

The above plot tells us that there are three resolved processes (blue, red, green) which are largely invariant to the MSM lag time.
The fourth ITS (cyan) is smaller than the lag time (black line, grey-shaded area);
it corresponds to a process which is faster than the lag time and, thus, is not resolved.
Since the implied timescales are, like the corresponding eigenvalues, sorted in decreasing order,
we know that all other remaining processes must be even faster.

## Error bars for the timescales

To compute error bars, pass the `errors=bayes` parameter:

In [ ]:
lags = [1, 2, 5, 10, 20, 50]
timescales = []
timescales_samples = []

for lag in tqdm(lags):
    counts_estimator = dt.markov.TransitionCountEstimator(lag, "effective")
    counts = counts_estimator.fit(dtrajs).fetch_model()
    counts = counts.submodel_largest()
    
    bmsm_estimator = dt.markov.msm.BayesianMSM()
    bmsm = bmsm_estimator.fit(counts).fetch_model()
    timescales.append(bmsm.prior.timescales(k=4))
    timescales_samples.append(bmsm.evaluate_samples('timescales', k=4).T)

pyemma.plots.plot_implied_timescales((lags, timescales, timescales_samples), units='ps')

## Effect of the discretization on the implied timescales

Let's look at the discretisation's influence on the ITSs:

In [ ]:
def its(k, data):
    cluster = dt.clustering.Kmeans(k, max_iter=50).fit(data[::10]).fetch_model()
    dtrajs = cluster.transform(data)
    
    lags = [1, 2, 5, 10, 20, 50]
    models = []
    msm_estimator = dt.markov.msm.MaximumLikelihoodMSM()
    estimator = dt.markov.msm.BayesianMSM()
    for lag in tqdm(lags):
        msm_estimator.lagtime = lag
        msm = msm_estimator.fit(dtrajs, count_mode="effective")
        bmsm = estimator.fit(msm).fetch_model()
        models.append(bmsm)
    
    ts = [x.prior.timescales(4) for x in models]
    ts_s = [x.evaluate_samples('timescales', k=4).T for x in models]
    return cluster, (lags, ts, ts_s)

In [ ]:
cluster_20, its_20 = its(20, data_concatenated)
cluster_50, its_50 = its(50, data_concatenated)
cluster_100, its_100 = its(100, data_concatenated)

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(12, 6))

pyemma.plots.plot_free_energy(*data_concatenated.T, ax=axes[0, 0], cbar=False)
axes[0, 0].scatter(*cluster_20.cluster_centers.T, s=15, c='k')
pyemma.plots.plot_implied_timescales(its_20, ax=axes[1, 0], units='ps')

pyemma.plots.plot_free_energy(*data_concatenated.T, ax=axes[0, 1], cbar=False)
axes[0, 1].scatter(*cluster_50.cluster_centers.T, s=15, c='k')
pyemma.plots.plot_implied_timescales(its_50, ax=axes[1, 1], units='ps')

pyemma.plots.plot_free_energy(*data_concatenated.T, ax=axes[0, 2], cbar=False)
axes[0, 2].scatter(*cluster_100.cluster_centers.T, s=15, c='k')
pyemma.plots.plot_implied_timescales(its_100, ax=axes[1, 2], units='ps')

fig.tight_layout()

## Estimating the maximum likelihood Markov model

In [ ]:
counts_estimator = dt.markov.TransitionCountEstimator(lagtime=10, count_mode='sliding')
counts = counts_estimator.fit(dtrajs).fetch_model().submodel_largest()

msm_estimator = dt.markov.msm.MaximumLikelihoodMSM()
msm = msm_estimator.fit(counts).fetch_model()

print(f'fraction of states used = {msm.state_fraction}')
print(f'fraction of counts used = {msm.count_fraction}')

In [ ]:
msm.timescales(k=4)

The state space can be restricted to largest connected set (`submodel_largest()`) or any other selection of states:

In [ ]:
counts = counts.submodel([0, 1, 3, 7])
print(f"States: {counts.states}, state symbols: {counts.state_symbols}")
msm =dt.markov.msm.MaximumLikelihoodMSM().fit(counts).fetch_model()

print(f'fraction of states used = {msm.state_fraction}')
print(f'fraction of counts used = {msm.count_fraction}')

And restricted even further, always based on the _states_ of the current count model.

In [ ]:
counts = counts.submodel([0, 3])
print(f"States: {counts.states}, state symbols: {counts.state_symbols}")
msm = dt.markov.msm.MaximumLikelihoodMSM().fit(counts).fetch_model()

print(f'fraction of states used = {msm.state_fraction}')
print(f'fraction of counts used = {msm.count_fraction}')

## Estimating the Bayesian Markov model

In [ ]:
count_estimator = dt.markov.TransitionCountEstimator(lagtime=10, count_mode='effective')
counts = count_estimator.fit(dtrajs).fetch_model()
counts = counts.submodel_largest()
bayesian_msm_estimator = dt.markov.msm.BayesianMSM()
bayesian_msm = bayesian_msm_estimator.fit(counts).fetch_model()

In [ ]:
stats = bayesian_msm.gather_stats('timescales', k=3)
stats.L, stats.R

## The Chapman-Kolmogorov test

To see whether our model satisfies Markovianity, we perform (and visualize) a Chapman-Kolmogorow (CK) test.
Since we aim at modeling the dynamics between metastable states rather than between microstates, this will be conducted in the space of metastable states.
The latter are identified automatically using PCCA++ (which is explained later).
We usually choose the number of metastable states according to the implied timescales plot by identifying a gap between the ITS.

In [ ]:
test_estimator = dt.markov.msm.MaximumLikelihoodMSM(lagtime=10)
test_estimator.fit(dtrajs);

In [ ]:
ck_validator = test_estimator.chapman_kolmogorov_validator(n_metastable_sets=4, mlags=10)
ck_test = ck_validator.fit(dtrajs).fetch_model()

In [ ]:
pyemma.plots.plot_cktest(ck_test, units='ps');

In [ ]:
count_estimator = dt.markov.TransitionCountEstimator(lagtime=10, count_mode='effective')
counts = count_estimator.fit(dtrajs).fetch_model().submodel_largest()
bmsm_estimator = dt.markov.msm.BayesianMSM()
bmsm_estimator.fit(counts);

In [ ]:
ck_val = bmsm_estimator.chapman_kolmogorov_validator(4, mlags=10)
cktest = ck_val.fit(dtrajs, progress=tqdm).fetch_model()

In [ ]:
pyemma.plots.plot_cktest(cktest, units='ps');

## Persisting and restoring estimators

In [ ]:
import pickle

with open('cluster_50.pkl', 'wb') as f:
    pickle.dump(cluster_50, f)
    
with open('cluster_50.pkl', 'rb') as f:
    cluster_50_restored = pickle.load(f)
    
print(cluster_50_restored.n_clusters)

In [ ]:
with open('msm.pkl', 'wb') as f:
    pickle.dump(msm, f)
with open('msm.pkl', 'rb') as f:
    msm_restored = pickle.load(f)
    
print(f"Timescales {msm.timescales()}, restored {msm_restored.timescales()}")

## Hands-on

#### Exercise 1

Load the heavy atom distances into memory, perform PCA and TICA (`lag=3`) with `dim=2`,
then discretize with $100$ $k$-means centers and a stride of $10$. Compare the two discretizations be generating implied timescale plots for both of them.

In [ ]:
from sklearn.decomposition import PCA

feat =  #FIXME
feat. #FIXME
data =  #FIXME

pca_estimator = PCA(2)
pca = #FIXME
tica = #FIXME

pca_concatenated = np.concatenate(pca.get_output())
tica_concatenated = #FIXME

cls_pca_estimator = dt.clustering.Kmeans(100, max_iter=50)
cls_pca = #FIXME
cls_tica = #FIXME

dtrajs_pca = [cls_pca.transform(pca.transform(traj)) for traj in data]
dtrajs_tica = # FIXME

def its(dtrajs, lags):
    models = []
    estimator = dt.markov.msm.BayesianMSM()
    for lag in tqdm(lags):
        counts_estimator = dt.markov.TransitionCountEstimator(lag, "effective")
        counts = # FIXME
        bayesian_msm = # FIXME
        models.append(bmsm)
    
    timescales = # FIXME
    timescales_samples = # FIXME
    return lags, timescales, timescales_samples

    
its_pca = its(dtrajs_pca, lags=[1, 2, 5, 10, 20, 50])
its_tica = #FIXME

###### Solution

In [ ]:
from sklearn.decomposition import PCA

feat = pyemma.coordinates.featurizer(pdb)
pairs = feat.pairs(feat.select_Heavy())
feat.add_distances(pairs, periodic=False)
data = pyemma.coordinates.load(files, features=feat)

pca = PCA(2)
pca.fit(np.concatenate(data))

tica_estimator = dt.decomposition.TICA(lagtime=3, dim=2)
tica = tica_estimator.fit(data).fetch_model()

pca_concatenated = np.concatenate([pca.transform(traj) for traj in data])
tica_concatenated = np.concatenate([tica.transform(traj) for traj in data])

cls_pca = dt.clustering.Kmeans(100, max_iter=50).fit(pca_concatenated).fetch_model()
cls_tica = dt.clustering.Kmeans(100, max_iter=50).fit(tica_concatenated).fetch_model()

def its(dtrajs, lags):
    models = []
    estimator = dt.markov.msm.BayesianMSM()
    for lag in tqdm(lags):
        counts_estimator = dt.markov.TransitionCountEstimator(lag, "effective")
        counts = counts_estimator.fit(dtrajs).fetch_model().submodel_largest()
        bmsm = estimator.fit(counts).fetch_model()
        models.append(bmsm)
    
    timescales = [x.prior.timescales(4) for x in models]
    timescales_samples = [x.evaluate_samples('timescales', k=4).T for x in models]
    return lags, timescales, timescales_samples


dtrajs_pca = [cls_pca.transform(pca.transform(traj)) for traj in data]
dtrajs_tica = [cls_tica.transform(tica.transform(traj)) for traj in data]
its_pca = its(dtrajs_pca, lags=[1, 2, 5, 10, 20, 50])
its_tica = its(dtrajs_tica, lags=[1, 2, 5, 10, 20, 50])

Let's visualize the ITS convergence for both projections:

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(12, 6))
pyemma.plots.plot_feature_histograms(pca_concatenated, ax=axes[0, 0])
pyemma.plots.plot_feature_histograms(tica_concatenated, ax=axes[1, 0])
axes[0, 0].set_title('PCA')
axes[1, 0].set_title('TICA')
pyemma.plots.plot_density(*pca_concatenated.T, ax=axes[0, 1], cbar=False, alpha=0.1)
axes[0, 1].scatter(*cls_pca.cluster_centers.T, s=15, c='C1')
axes[0, 1].set_xlabel('PC 1')
axes[0, 1].set_ylabel('PC 2')
pyemma.plots.plot_density(*tica_concatenated.T, ax=axes[1, 1], cbar=False, alpha=0.1)
axes[1, 1].scatter(*cls_tica.cluster_centers.T, s=15, c='C1')
axes[1, 1].set_xlabel('IC 1')
axes[1, 1].set_ylabel('IC 2')
pyemma.plots.plot_implied_timescales(its_pca, ax=axes[0, 2], units='ps')
pyemma.plots.plot_implied_timescales(its_tica, ax=axes[1, 2], units='ps')
axes[0, 2].set_ylim(1, 2000)
axes[1, 2].set_ylim(1, 2000)
fig.tight_layout()

Despite the fact that PCA yields a projection with some defined basins,
the ITS plot shows that only one "slow" process is resolved which is more than one order of magnitude too fast.

TICA does find three slow processes which agree (in terms of the implied timescales) with the backbone torsions example above.

We conclude that this PCA projection is not suitable to resolve the slow dynamics of alanine dipeptide and we will continue to estimate/validate the TICA-based projection.

#### Exercise 2

Estimate a Bayesian MSM at lag time $10$ ps and perform/show a CK test for four metastable states.

In [ ]:
counts_estimator = dt.markov.TransitionCountEstimator(lagtime=10, count_mode="effective")
counts = counts_estimator.fit(dtrajs_tica).fetch_model().submodel_largest()
bayesian_msm = # FIXME
pyemma.plots. #FIXME

###### Solution

In [ ]:
counts_estimator = dt.markov.TransitionCountEstimator(lagtime=10, count_mode="effective")
counts = counts_estimator.fit(dtrajs_tica).fetch_model().submodel_largest()

estimator = dt.markov.msm.BayesianMSM()
estimator.fit(counts)
validator = estimator.chapman_kolmogorov_validator(4, mlags=10)
cktest = validator.fit(dtrajs_tica).fetch_model()
pyemma.plots.plot_cktest(cktest, units='ps');

We again see a good agreement between model prediction and re-estimation.

## Wrapping up
In this notebook, we have learned how to estimate a regular or Bayesian MSM from discretized molecular simulation data with `pyemma` and how to perform basic model validation.
In detail, we have selected a suitable lag time by using
- `pyemma.msm.its()` to obtain an implied timescale object and
- `pyemma.plots.plot_implied_timescales()` to visualize the convergence of the implied timescales.

We then have used
- `pyemma.msm.estimate_markov_model()` to estimate a regular MSM,
- `pyemma.msm.bayesian_markov_model()` to estimate a Bayesian MSM,
- the `timescales()` method of an estimated MSM object to access its implied timescales,
- the `cktest()` method of an estimated MSM object to perform a Chapman-Kolmogorow test, and
- `pyemma.plots.plot_cktest()` to visualize the latter.